<a href="https://colab.research.google.com/github/venkat2ram/ML_From_Scratch/blob/master/NeuralNet_From_Scratch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np


In [0]:
def init_params(layer_dims):
  np.random.seed(3)
  L=len(layer_dims)
  params={}
  for l in range(1,L):
    params['w'+str(l)]=np.random.randn(layer_dims[l],layer_dims[l-1])/0.01
    params['b'+str(l)]=np.zeros((layer_dim[l],1))
  return params


In [0]:
def sigmoid(Z):
  A=1/(1+np.exp(np.dot(-1,Z)))
  cache=(Z)
  return A,cache

In [0]:
def forward_prop(X,params):
  A=X
  caches=[]
  L=len(params)//2
  for l in range(1,L+1):
    A_prev=A
    Z=np.dot(params['w'+str(l)],A_prev)+params['b'+str(l)]
    linear_cache=(A_prev,params['w'+str(l)],params['b'+str(l)])
    A,activation_cache=sigmoid(Z)
    cache=(linear_cache,activation_cache)
    caches.append(cache)
  return A,caches   


In [0]:
def cost_function(A, Y):
  m=Y.shape[1]
  cost=(-1/m)*np.sum(np.dot(np.log(A),Y.T)+np.dot(np.log(1-A),1-Y.T))
  return cost

In [0]:
def one_layer_backward(dA,cache):
  linear_cache, activation_cache=cache
  
  Z=activation_cache
  dZ=dA*sigmoid(Z)*(np.ones(d2.shape)-sigmoid(Z))

  A_prev,W,b=linear_cache
  m=A_prev.shape[1]
  dW=(1/m)*(np.dot(dZ,A_prev.T))
  dB=(1/m)*(np.sum(dZ,axis=1,keepdims=True))
  dA_prev=np.dot(W.T,dZ)

  return dA_prev,dW,dB

In [0]:
def backprop(AL,Y,caches):
  grads={}
  L=len(caches)
  dA=-1*(np.divide(Y,AL)+np.divide(1-Y,1-AL))

  grads['dA'+str(L-1)],grads['dW'+str(L-1)],grads['dB'+str(L-1)]=one_layer_backward(dA,caches[L-1])

  for l in reversed(range(L-1)):
    grads['dA'+str(l)],grads['dW'+str(l)],grads['dB'+str(l)]=one_layer_backward(grads['dA'+str(l+1)],cache[l])
  return grads
  

**Rough**

In [29]:
a=np.array([[0.2,0.2,0.3],[0.3,0.4,0.5],[0.4,0.3,0.5]])
b=np.array([[0.3,0.4,0.5],[0.6,0.7,0.8]])
cost_function(a,b)


4.742705654642286

In [31]:
a.shape[1]

3

In [0]:
layer_dim=[3,4,4,2]

In [0]:
params=init_params(layer_dim)

In [0]:
y_hat,cache=forward_prop(a,params)

In [0]:
dA=-1*np.divide(b,y_hat)+np.divide(1-b,1-y_hat)

In [74]:
one_layer_backward(dA,cache[2])

(array([[[ 9.77124902e+01,  1.17842087e+02,  1.37971684e+02],
         [-1.25258735e+60, -1.46135191e+60, -1.67011647e+60]],
 
        [[ 4.44859363e+01,  5.30504127e+01,  6.16148890e+01],
         [-6.30678552e+59, -7.35791644e+59, -8.40904736e+59]],
 
        [[ 2.58742777e+01,  2.64613760e+01,  2.70484742e+01],
         [-8.09234277e+59, -9.44106657e+59, -1.07897904e+60]],
 
        [[ 8.58616669e+01,  9.02913908e+01,  9.47211147e+01],
         [-2.43554068e+60, -2.84146413e+60, -3.24738758e+60]]]),
 array([[ -76.88363503,  -23.00307223,   74.50562664,  197.61107831],
        [-124.4123329 ,  -62.64169112,  -80.37660946, -241.90831732]]),
 array([[0.],
        [0.]]))

In [38]:
y_hat

array([[4.07250435e-34, 4.07250435e-34, 4.07250435e-34],
       [9.29844535e-55, 9.29844535e-55, 9.29844535e-55]])

In [39]:
cache

[((array([[0.2, 0.2, 0.3],
          [0.3, 0.4, 0.5],
          [0.4, 0.3, 0.5]]),
   array([[ 178.86284734,   43.65098505,    9.64974681],
          [-186.34927034,  -27.73882025,  -35.47589793],
          [  -8.27414815,  -62.70006768,   -4.3818169 ],
          [ -47.72180304, -131.38647534,   88.46223805]]),
   array([[0.],
          [0.],
          [0.],
          [0.]])),
  array([[ 52.72776371,  56.12788753,  80.30922013],
         [-59.78185931, -59.00815155, -87.51214019],
         [-22.21757669, -28.04940177, -36.02318673],
         [-13.57540799, -35.56027933, -35.77865955]])),
 ((array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
          [1.08910157e-26, 2.36094242e-26, 9.86189595e-39],
          [2.24403197e-10, 6.58111667e-13, 2.26635940e-16],
          [1.27137793e-06, 3.60053398e-16, 2.89418032e-16]]),
   array([[  88.13180422,  170.95730637,    5.00336422,  -40.46774146],
          [ -54.53599476, -154.64773156,   98.23674343, -110.10676301],
          [-118.504

In [0]:
params['w1']

In [0]:
a

In [0]:
np.dot(params['w1'],a)

In [0]:
sigmoid(2)

In [0]:
c1,c2=cache[2]

In [46]:
c2

array([[ -76.88363503,  -76.88363503,  -76.88363503],
       [-124.4123329 , -124.4123329 , -124.4123329 ]])

In [47]:
dA

array([[-7.36647463e+32, -9.82196618e+32, -1.22774577e+33],
       [-6.45269158e+53, -7.52814018e+53, -8.60358877e+53]])

In [49]:
dA*sigmoid(c2)*sigmoid(1-c2)

array([[[-3.00000000e-01, -4.00000000e-01, -5.00000000e-01],
        [-6.00000000e-01, -7.00000000e-01, -8.00000000e-01]],

       [[ 4.41102805e+36,  5.88137073e+36,  7.35171341e+36],
        [ 1.00680320e+58,  1.17460374e+58,  1.34240427e+58]]])

In [67]:
d1*sigmoid(d2)*(np.ones(d2.shape)-sigmoid(d2))

array([[[-3.00000000e-01, -4.00000000e-01, -5.00000000e-01],
        [-6.00000000e-01, -7.00000000e-01, -8.00000000e-01]],

       [[ 4.41102805e+36,  5.88137073e+36,  7.35171341e+36],
        [ 1.00680320e+58,  1.17460374e+58,  1.34240427e+58]]])

In [64]:
sigmoid(d2)

(array([[4.07250435e-34, 4.07250435e-34, 4.07250435e-34],
        [9.29844535e-55, 9.29844535e-55, 9.29844535e-55]]),
 array([[ -76.88363503,  -76.88363503,  -76.88363503],
        [-124.4123329 , -124.4123329 , -124.4123329 ]]))

In [66]:
np.ones(d2.shape)-sigmoid(d2)

array([[[  1.        ,   1.        ,   1.        ],
        [  1.        ,   1.        ,   1.        ]],

       [[ 77.88363503,  77.88363503,  77.88363503],
        [125.4123329 , 125.4123329 , 125.4123329 ]]])

In [83]:
for l in reversed(range(4)):
  print(l)

3
2
1
0
